In [1]:
import pandas as pd

In [2]:
master02_10 = pd.read_csv('./data/MASTER2002_2010.csv',index_col='Unnamed: 0')
master10_20 = pd.read_csv('./data/MASTER2010_2020.csv',index_col='Unnamed: 0')

In [5]:
master = pd.concat([master02_10,master10_20])
master['HomeOrAway'] = master['HomeOrAway'].astype(str)
master = master[~master.HomeOrAway.str.contains('@')]
master.drop(['PlayersURL','VegasLink','GetBoxScoreURL','Weather','BOXSCORE', 'HomeOrAway','Links'], axis=1, inplace=True)
master = master.reset_index(drop=True)

In [10]:
#Drop bobby boucher and forest gump
homeQBTD = pd.pivot_table(master, 'Home_QB_Passing_TD', 'Home_QB_Player', 'Team_Name').sum()
homeRBTD = pd.pivot_table(master, 'H_RB1_Rushing_TD', 'H_RB1_Player', 'Team_Name').sum()
homeWRTD = pd.pivot_table(master, 'H_WR1_Receiving_TD', 'H_WR1_Player', 'Team_Name').sum()
awayQBTD = pd.pivot_table(master, 'Away_QB_Passing_TD', 'Away_QB_Player', 'Team_Name').sum()
awayRBTD = pd.pivot_table(master, 'A_RB1_Rushing_TD', 'A_RB1_Player', 'Team_Name').sum()
awayWRTD = pd.pivot_table(master, 'A_WR1_Receiving_TD', 'A_WR1_Player', 'Team_Name').sum()
homeRBTD = pd.pivot_table(master, 'H_RB1_Rushing_TD', 'H_RB1_Player', 'Team_Name').sum()
# awayRBTD = pd.pivot_table(master, 'A_RB1_Rushing_TD', 'Team_Name','A_RB1_Player').sum()
homeWRTD





Team_Name
Arizona Cardinals       15.210067
Atlanta Falcons         26.671717
Baltimore Ravens        21.939899
Buffalo Bills            8.846961
Carolina Panthers       18.537085
Chicago Bears           15.489165
Cincinnati Bengals      24.544528
Cleveland Browns         9.281622
Dallas Cowboys          24.258333
Denver Broncos          22.381549
Detroit Lions           20.575817
Green Bay Packers       32.835455
Houston Texans          21.994444
Indianapolis Colts      24.907631
Jacksonville Jaguars    17.836851
Kansas City Chiefs      22.429600
Miami Dolphins          10.971817
Minnesota Vikings       15.424603
New England Patriots    36.163492
New Orleans Saints      24.627145
New York Giants         24.669733
New York Jets           18.369181
Oakland Raiders         10.940110
Philadelphia Eagles     47.023793
Pittsburgh Steelers     16.237143
San Diego Chargers      14.283117
San Francisco 49ers     16.641583
Seattle Seahawks        24.722222
St. Louis Rams          16.582479
Tamp

In [ ]:
# Team Home wins by team over all years
HomeWins = master['W_L'].groupby(master['Team_Name']).describe().sort_values(by='count', ascending=False)
AwayWins = master['W_L'].groupby(master['Home_QB_Player']).describe().sort_values(by='count', ascending=False)
group = master['W_L'].groupby([master['Team_Name'],master['Team_Year']]).count()
group

In [8]:
col = [x for x in master.columns]
col

['Week',
 'Day',
 'Date',
 'Time_ET',
 'W_L',
 'OT',
 'Home_Record',
 'Opp',
 'Home_Score',
 'Away_Score',
 'Home_TotYd',
 'Away_TotYd',
 'Away_RushY',
 'Expected Points_Home',
 'Expected Points_Away',
 'Expected Points_Sp. Tms',
 'Team_Name',
 'Team_Year',
 'Date_y',
 'Home_QB_Player',
 'Home_QB_Team',
 'Home_QB_Passing_Cmp',
 'Home_QB_Passing_Att',
 'Home_QB_Passing_Yds',
 'Home_QB_Passing_TD',
 'Home_QB_Passing_Int',
 'Home_QB_Passing_Sk',
 'Home_QB_Passing_Yds.1',
 'Home_QB_Passing_Lng',
 'Home_QB_Passing_Rate',
 'Home_QB_Rushing_Att',
 'Home_QB_Rushing_Yds',
 'Home_QB_Rushing_TD',
 'Home_QB_Rushing_Lng',
 'Away_QB_Player',
 'Away_QB_Team',
 'Away_QB_Passing_Cmp',
 'Away_QB_Passing_Att',
 'Away_QB_Passing_Yds',
 'Away_QB_Passing_TD',
 'Away_QB_Passing_Int',
 'Away_QB_Passing_Sk',
 'Away_QB_Passing_Yds.1',
 'Away_QB_Passing_Lng',
 'Away_QB_Passing_Rate',
 'Away_QB_Rushing_Att',
 'Away_QB_Rushing_Yds',
 'Away_QB_Rushing_TD',
 'Away_QB_Rushing_Lng',
 'H_RB1_Player',
 'H_RB1_Team',
 'H

In [16]:
master = pd.read_csv('./data/MASTER2002_2010.csv',index_col='Unnamed: 0')
test = master[-1:]

In [17]:
col = [x for x in master.columns]
ou = test['Home_Score'] + test['Away_Score']

In [22]:
# print(test['Home_Score'],test['Away_Score'], )
print(f"Home Score {test['Home_Score']}, Away Score {test['Away_Score']} for a total of {ou} and the overunder was {test['Over/Under']}")


Home Score 5129    14.0
Name: Home_Score, dtype: float64, Away Score 5129    17.0
Name: Away_Score, dtype: float64 for a total of 5129    31.0
dtype: float64 and the overunder was 5129    44.5(under)
Name: Over/Under, dtype: object


In [24]:
col
dropcols = ('PlayersURL','VegasLink','GetBoxScoreURL','Weather')

In [ ]:
# 2002-2005  06-20
#madden ranking

html = get(url)
soup = Soup(html)
dfstats = pd.read_html(str(soup.find('table')[2]))[0]